In [9]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [10]:

# Define the data directory
data_dir = 'C:/Users/yadav/OneDrive/Documents/Coding/ImageProcessing/raw-imgModified'


In [ ]:
# Load the dataset
images, labels = load_dataset(data_dir)

# Convert the labels to categorical data
label_names = np.unique(labels)

In [ ]:
import random
import matplotlib.pyplot as plt

# Define a function to display random images with class names and sizes
def display_random_images(data_dir, label_names, num_images=10):
    # Get a list of all image file paths in the data directory
    image_paths = glob.glob(os.path.join(data_dir, '*', '*.jpg'))
    # Select num_images random image file paths
    image_paths = random.sample(image_paths, num_images)
    # Create a figure and axes objects
    fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(12, 6))
    # Set the title of the figure
    fig.suptitle('Random Images from the Dataset', fontsize=16)
    # Iterate over the randomly selected image file paths
    for i, image_path in enumerate(image_paths):
        # Read the image from the file path
        image = Image.open(image_path)
        # Get the class name from the directory name of the image
        class_name = os.path.basename(os.path.dirname(image_path))
        # Get the size of the image
        size = image.size
        # Choose the subplot to display the image and its metadata
        row = i // 5
        col = i % 5
        axs[row, col].imshow(image)
        axs[row, col].set_title(f'Class: {label_names.index(class_name)} - {class_name}\nSize: {size}')
        axs[row, col].axis('off')
    # Show the figure
    plt.show()

In [ ]:
# Call the function to display 10 random images with class names and sizes
display_random_images(data_dir, label_names)

In [16]:
# Define the batch size and number of epochs
batch_size = 32
num_epochs = 2

# Define the input shape and number of classes
input_shape = (200, 200, 3)
num_classes = 10

In [12]:
# Define the data generator for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 20946 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [17]:
# Define the InceptionResNetV2 model
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape
)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [18]:
# Train the model
history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator
)

Epoch 1/2
417/655 [==================>...........] - ETA: 6:27 - loss: 0.3027 - accuracy: 0.9182

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(val_generator)
print(f'Test loss: {test_loss:.3f*100}')
print(f'Test accuracy: {test_acc:.3f*100}')


In [ ]:

# Plot the training and validation accuracy and loss
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [ ]:

# Generate predictions on the test set and plot the confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

labels_test = val_generator.classes
images_test = np.concatenate([val_generator.next()[0] for i in range(val_generator.samples // batch_size)])

y_pred = np.argmax(model.predict(images_test), axis=-1)


In [ ]:

# Plot the confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)
plt.figure(figsize=(20, 20))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()